## Estrategia Medias Móviles

In [1]:
import sys
sys.path.append("../config_files")
#sys.path.remove("../config_files")
print(sys.path)

['/Users/fernando/Desktop/TradingBot/trading-bot', '/opt/anaconda3/envs/trading/lib/python39.zip', '/opt/anaconda3/envs/trading/lib/python3.9', '/opt/anaconda3/envs/trading/lib/python3.9/lib-dynload', '', '/opt/anaconda3/envs/trading/lib/python3.9/site-packages', '/opt/anaconda3/envs/trading/lib/python3.9/site-packages/IPython/extensions', '/Users/fernando/.ipython', '../config_files']


In [2]:
import mysql.connector
import pandas as pd
import numpy as np
import plotly.offline 
#from plotly.offline import download_plotlyjs, plot, iplot, init_notebook_mode
import plotly.graph_objs as go

%matplotlib inline

ModuleNotFoundError: No module named 'plotly'

In [2]:
import plotly.offline
import plotly.graph_objs as go

plotly.offline.plot({"data": [go.Scatter(x=[1, 2, 3, 4], y=[4, 3, 2, 1])],
                     "layout": go.Layout(title="hello world")},
                     image='jpeg', image_filename='test')

ModuleNotFoundError: No module named 'plotly'

In [3]:
import plotly.graph_objects as go
import numpy as np
np.random.seed(1)

N = 100
x = np.random.rand(N)
y = np.random.rand(N)
colors = np.random.rand(N)
sz = np.random.rand(N) * 30

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x,
    y=y,
    mode="markers",
    marker=go.scatter.Marker(
        size=sz,
        color=colors,
        opacity=0.6,
        colorscale="Viridis"
    )
))

fig.show()

ModuleNotFoundError: No module named 'plotly'

### Leer dtos de alguna DB

In [2]:
def load_bitso_data_top500():
    
    conn = mysql.connector.connect(user='', database='')
    cursor = conn.cursor()
    query = """SELECT
        *
    FROM
        bitso_api.bitso_mxn
        
    ORDER BY
        tid DESC
    LIMIT 5000;"""
    extraction = pd.read_sql(query.conn)
    return extraction

datos = load_bitso_data_top500(CONFIG_MYSQL)

NameError: name 'CONFIG_MYSQL' is not defined

In [3]:
datos.columns

NameError: name 'datos' is not defined

In [4]:
datos.head()

NameError: name 'datos' is not defined

In [5]:
datos.tail()

NameError: name 'datos' is not defined

In [6]:
datos = datos.reindex(index=datos.index[::-1])
datos.reset_index(inplace=True,drop=True)

NameError: name 'datos' is not defined

In [7]:
datos.head()

NameError: name 'datos' is not defined

In [ ]:
datos.dtypes

In [ ]:
datos.price.plot(kind='line')

## Calcular Medias Móviles

In [ ]:
def sma(df,d):
    c = df.rolling(d).mean()
    return c.dropna()

In [ ]:
datos['mv200'] = sma(datos.price,10)
datos['mv160'] = sma(datos.price,60)

In [ ]:
datos.head(30)

In [ ]:
columnas = ['prrice', 'mv20', 'mv160']

fig = go.Figure()
for columna in columnas:
    fig.add_trace(go.Sctter(x=datos.index, y = datos[columna],
                     mode = 'lines',
                     name = columna))
fig.update_layout(tamplate='plotly_dark')

iplot(fig)

## Creamos lógica de estretegia

In [ ]:
datos['alpha'] = datos['mv20'] - datos['mv160']

In [ ]:
datos['alpha']

In [ ]:
datos['alpha'].plot()

In [ ]:
datos['alpha_bin'] = datos['alpha'].apply(np.sign)

In [ ]:
datos['alpha_bin']

In [ ]:
datos['alpha_bin'].plot()

In [ ]:
datos['alpha_bin'].value_counts()

In [ ]:
datos['alpha_trade_long'] = ((datos['alpha_bin'] == 1) & (datos['alpha_bin'].shift(1) == -1) &
                             (datos['alpha_bin'].shift(2) == -1) & (datos['alpha_bin'].shift(3) == -1))

datos['alpha_trade_short'] = ((datos['alpha_bin'] == -1) & (datos['alpha_bin'].shift(1) == 1) &
                             (datos['alpha_bin'].shift(2) == 1) & (datos['alpha_bin'].shift(3) == 1))

In [ ]:
datos['alpha_trade_long'].value_counts()

In [ ]:
datos['alpha_trade_short'].value_counts()

In [ ]:
datos['alpha_trade_compra'] = np.where(datos['alpha_trade_long'] == True, datos['mv20'], np.nan)
datos['alpha_trade_venta'] = np.where(datos['alpha_trade_short'] == True, datos['mv20'], np.nan)

In [ ]:
columnas = ['mv20','mv160','alpha_trade_compra','alpha_trade_venta']

fig = go.Figure()
for columna in columnas:
    if columna == 'alpha_trade_compra':
        fig.add_trace(go.Scatter(x=dato.index,y=datos[columna],
                                mode = 'markers',
                                name=columna,
                                marker = dict(color='green') ))
        
    if columna == 'alpha_trade_venta':
        fig.add_trace(go.Scatter(x=datos.index, y=datos[columna],
                                mode = 'markers',
                                name=columna,
                                marker=dict(color='red') ))
    else:
        fig.add_trace(go.Scatter(x=datos.index,y=datos[columna],
                                mode = 'lines',
                                name=columna))
        
fig.update_layout(template='plotly_dark')

iplot(fig)
                      
                      